# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
from scipy.stats import norm
from scipy.optimize import minimize

def Blacks_formula(sigma, maturity, strike, F, discount, price='Call'):
    '''
    Inputs: Volatility, Maturity, strike, Forward, discount, and type of option 'Call' or 'Put'
    Returns option's price.
    '''
    vol=sigma
    T=maturity
    K=strike
    Z=discount
    d1=(np.log(F/K)+(((vol**2)*T)/2))/(vol*(T**(1/2)))
    d2=d1-(vol*(T**(1/2)))
    if price=='Call':
        return Z*((F*norm.cdf(d1))-(K*norm.cdf(d2)))
    if price=='Put':
        return Z*((K*norm.cdf(-d2))-(F*norm.cdf(-d1)))

def price_cap_floor(sigma, maturity, strike, F, discount, instrument='Cap', n=4):
    if np.isnan(sigma):
        return np.nan
    if instrument=='Cap':
        price='Call'
    if instrument == 'Floor':
        price='Put'
    tp=0
    for i in range(2,int(n*maturity)+1):
        tp+=Blacks_formula(sigma, (i/n)-(1/n), strike, F[i-1], discount[i-1], price)
    return (100/n)*tp

def forward_vols_from_instrument_prices(cap_prices, maturities, strikes, Fs, discounts, price='Call'):
    cap_prices=np.array(cap_prices)
    maturities=np.array(maturities)
    strikes=np.array(strikes)
    Fs=np.array(Fs)
    discounts=np.array(discounts)

    n=1/(maturities[1]-maturities[0])
    res=np.zeros((len(cap_prices)))
    
    for i, cap_price in enumerate(cap_prices):
        if np.isnan(cap_price):
            res[i]=np.nan
        else:
            tp=0
            for ii in range(i):
                if not np.isnan(res[ii]):
                    tp+=Blacks_formula(res[ii],maturities[ii-1],strikes[i],Fs[ii],discounts[ii],price)
            ttp=cap_price-(tp*100/n)
            def helper(sigma):
                return ((Blacks_formula(sigma, maturities[i-1], strikes[i], Fs[i], discounts[i], price)*100/n) - (ttp))**2
            res[i]=minimize(helper,[0.05], tol=1e-8).x[0]
    return res

def continuous_from_discount(discount, maturity):
    '''Returns the continuous rate from the discount factor and the maturity'''
    return -np.log(discount)/maturity

def compounded_from_discount(discount, maturity, freq=2):
    '''Returns the compounded rate from the discount factor and the maturity'''
    return ((1/discount)**(1/(maturity*freq))-1)*freq

def discount_from_continuous(continuous, maturity):
    '''Returns the discount factor from the continuous rate and the maturity'''
    return np.exp(-continuous*maturity)

def discount_from_compounded(compounded, maturity, freq=2):
    '''Returns the discount factor from the compounded rate and the maturity'''
    return (1+compounded/freq)**(-maturity*freq)

def display_tree(periods, levels, attribute):
    temp=pd.DataFrame()
    for i in range(1,len(levels)+1):
        temp_level=[]
        for j in range(len(levels[-i])):
            temp_level.append(getattr(levels[-i][j],attribute))
        temp[round(periods[-i],2)]=temp_level+['']*(len(levels[-1])-len(temp_level))
    temp.sort_index(axis=1, inplace=True)
    temp.columns=temp.columns.astype(str)
    return temp.style.format("{:.2}")
        
def display_tree_no_attribute(periods, levels):
    temp=pd.DataFrame()
    for i in range(1,len(levels)+1):
        temp_level=[]
        for j in range(len(levels[-i])):
            temp_level.append(levels[-i][j])
        temp[round(periods[-i],2)]=temp_level+['']*(len(levels[-1])-len(temp_level))
    temp.sort_index(axis=1, inplace=True)
    temp.columns=temp.columns.astype(str)
    return temp
    
class tree_convention:
    UP=1
    DOWN=-1
    ORIGIN=0
    TERMINAL=2
    DUMMY=3
    
class interest_node():
    def __init__(self, period, rate, state=tree_convention.DUMMY):
        self.period = period
        self.rate = rate
        self.state = state

    def __repr__(self):
        return 'rate: {:.2%}\nperiod: {}\nstate: {}'.format(self.rate,self.period,self.state)
    
class Interest_Tree():
    def __init__(self, periods, rates=None):
        self.periods = periods
        self.rates = rates
        self.levels = []

    def tree_from_discounts_theta_sigma(self,discounts,theta,sigma,freq=4):
        self.levels.append([interest_node(0,continuous_from_discount(discounts[0],1/freq),tree_convention.ORIGIN)])
        for i in range(1,len(self.periods)):
            tlevel=[]
            for j in range(len(self.levels[i-1])):
                trate=self.levels[i-1][j].rate
                tlrate=np.log(trate*100)
                ttheta=theta[i-1]
                tsigma=sigma[i-1]
                tlrateup=tlrate+(ttheta*(1/freq))+(tsigma*np.sqrt(1/freq))
                trateup=np.exp(tlrateup)/100                   
                node=interest_node(self.periods[i],trateup)
                tlevel.append(node)
                if j == i-1:
                    tlratedown=tlrate+(ttheta*(1/freq))-(tsigma*np.sqrt(1/freq))
                    tratedown=np.exp(tlratedown)/100
                    node=interest_node(self.periods[i],tratedown)
                    tlevel.append(node)
            self.levels.append(tlevel)

    def __repr__(self):
        tree_str = 'Rates_Tree [%]:\n'
        for level_index, level in enumerate(self.levels):
            level_str = '  ' * level_index
            for node in level:
                node_str = '{:.2f}'.format(
                    node.rate*100)
                level_str += node_str + ' '
            tree_str += 'period: {} ||'.format(node.period) + level_str.strip() + '\n'
        return tree_str.strip()
                

class bond_node():
    def __init__(self, period, price, state, maturity,expected_price=None):
        self.period = period
        self.price = price
        self.state = state
        self.maturity = maturity
        self.expected_price=expected_price
        self.rn_probUP=1/2
        self.rn_probDOWN=1/2

    def __repr__(self):
        return 'period: {}\nstate: {}\nprice: {:.2f}\nmaturity: {}'.format(self.period, self.state, self.price, self.maturity)

class Binary_Bond_Tree():
    def __init__(self, periods=None, price=None, face_value=100, interest_tree=None, maturity=None):
        self.periods = np.array(periods)
        if len(price)<=1:
            self.price=price
        else:
            self.price=np.array(price)
        self.face_value = face_value
        self.levels = []
        self.interest_tree = interest_tree
        self.maturity = maturity
        self.thetas=[]
        self.sigmas=np.zeros(())

    def tree_from_sigmas(self,sigmas,periods=None, prices=None):
        sigmas=np.array(sigmas)
        if periods is not None:
            self.periods=periods
        freq=1/(self.periods[1]-self.periods[0])
        if prices is not None:
            self.price=prices
        self.sigmas=sigmas
        for i in range(len(self.periods)-2):
            def helper(theta):
                ttree=Interest_Tree(self.periods[:i+2])
                ttree.tree_from_discounts_theta_sigma(self.price[:i+2]/100,np.append(self.thetas[:i],theta),sigmas[:i+2],freq)
                tjprices=[]
                for j in range(len(ttree.levels[-1])):
                    tjprices.append(self.face_value*np.exp(-ttree.levels[-1][j].rate*(1/freq)))
                for j in range(2,len(ttree.levels)+1):
                    tkprices=[]
                    for k in range(len(ttree.levels[-j])):
                        average=(tjprices[k]+tjprices[k+1])/2
                        tkprice=average*np.exp(-ttree.levels[-j][k].rate*(1/freq))
                        tkprices.append(tkprice)
                    tjprices=tkprices
                if len(tjprices)!=1:
                    print('Error', len(tjprices))
                return (tjprices[0]-self.price[i+1])**2
            theta=minimize(helper,[0.5], tol=1e-8).x[0]
            self.thetas.append(theta)
        ttree=Interest_Tree(self.periods[:-1])
        ttree.tree_from_discounts_theta_sigma(self.price/100,self.thetas,sigmas,freq)
        self.interest_tree=ttree

    def __repr__(self):
        tree_str = 'Binary_Bond_tree:\n'
        tree_str += '                               Maturity: {}\n'.format(self.maturity)
        for level_index, level in enumerate(self.levels[:-1]):
            level_str = '  ' * level_index
            for node in level:
                node_str = '[state: {}, price: {:.2f}]'.format(
                    node.state, node.price)
                level_str += node_str + '  '
            tree_str += 'period: {} '.format(node.period) + level_str.strip() + '\n'
        return tree_str.strip()

In [32]:
data=pd.read_excel('../data/swaption_vol_data.xlsx')
data

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

In [33]:
rates_cap=pd.read_excel('../data/cap_quotes_2024-02-20.xlsx',sheet_name='cap')
rates_sofr=pd.read_excel('../data/cap_quotes_2024-02-20.xlsx',sheet_name='sofr')

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.


In [34]:
rate_data=pd.read_excel('../data/cap_curves_2024-02-20.xlsx')
rate_data

,tenor,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
0,0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
1,0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
2,0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
3,1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
4,1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247
5,1.50,0.046235,0.046141,0.933499,0.039030,0.292615,0.380930
6,1.75,0.045059,0.044939,0.924774,0.037738,0.315878,0.388953
7,2.00,0.044133,0.043994,0.916212,0.037382,0.331443,0.386643
8,2.25,0.043173,0.043011,0.908230,0.035151,0.340919,0.376247
9,2.50,0.042461,0.042283,0.900188,0.035738,0.345916,0.363764


In [35]:
discount5=rate_data['discounts'].iloc[19]
discount1=rate_data['discounts'].iloc[3]

In [36]:
discount_swap=discount5/discount1
discount_swap

0.8640942156914205

In [37]:
swap_rate=compounded_from_discount(discount_swap,4,4)
swap_rate*100

3.668557489963309


## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.


In [38]:
data

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


In [39]:
vols=np.array(data.iloc[0,6:])/100
strikes=(np.array(data.columns[6:],dtype=float)/10000)+swap_rate

In [40]:
scale=np.sum(rate_data['discounts'].iloc[4:20])/4
scale

3.524129268625859

In [41]:
strikes[-5]

0.03668557489963309

In [42]:
swap_rate

0.03668557489963309

In [43]:
scale

3.524129268625859

In [44]:
prices=[]
for i in range(len(vols)):
    prices.append(100*Blacks_formula(vols[i],1,strikes[i],swap_rate,scale))
prices

[7.202086165489279,
 4.063798009429432,
 2.7381565055448984,
 2.173136435377561,
 1.6857037328339315,
 1.2796104825461665,
 0.9541191963114497,
 0.5121759203056722,
 0.14674956316667265]


## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [45]:
vols[-5]

0.3283

In [46]:
Swap1=prices[-5]
Swap2=100*Blacks_formula(vols[-5],
                         0.25,
                         compounded_from_discount(rate_data['discounts'].iloc[16]/rate_data['discounts'].iloc[0],4,4),
                         compounded_from_discount(rate_data['discounts'].iloc[16]/rate_data['discounts'].iloc[0],4,4),
                         np.sum(rate_data['discounts'].iloc[1:17])/4)
Swap3=100*Blacks_formula(vols[-5],
                         2,
                         compounded_from_discount(rate_data['discounts'].iloc[23]/rate_data['discounts'].iloc[7],4,4),
                         compounded_from_discount(rate_data['discounts'].iloc[23]/rate_data['discounts'].iloc[7],4,4),
                         np.sum(rate_data['discounts'].iloc[8:24])/4)

Swap0=100*Blacks_formula(vols[-5],
                         1,
                         compounded_from_discount(rate_data['discounts'].iloc[19]/rate_data['discounts'].iloc[3],4,4),
                         compounded_from_discount(rate_data['discounts'].iloc[19]/rate_data['discounts'].iloc[3],4,4),
                         np.sum(rate_data['discounts'].iloc[4:20])/4)

print(Swap0, Swap1, Swap2, Swap3)


1.6857037328339315 1.6857037328339315 0.9228018628376365 2.2593357836679555


***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

In [47]:
Tree=Binary_Bond_Tree(periods=np.arange(0,5.25,1/4),price=np.array(rate_data['discounts'].iloc[:20])*100)
Tree.tree_from_sigmas(np.array(rate_data['fwd vols'].bfill()))

In [48]:
print('Continuously Compounded')
display_tree(Tree.periods,Tree.interest_tree.levels,'rate')

Continuously Compounded


,0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0,2.25,2.5,2.75,3.0,3.25,3.5,3.75,4.0,4.25,4.5,4.75,5.0
0,0.052,0.055,0.056,0.058,0.057,0.06,0.065,0.072,0.077,0.09,0.11,0.13,0.14,0.16,0.2,0.24,0.28,0.33,0.41,0.49
1,,0.046,0.048,0.049,0.048,0.051,0.055,0.061,0.065,0.076,0.09,0.11,0.12,0.14,0.17,0.2,0.24,0.28,0.34,0.42
2,,,0.04,0.041,0.041,0.043,0.046,0.052,0.055,0.065,0.076,0.09,0.1,0.12,0.14,0.17,0.2,0.24,0.29,0.35
3,,,,0.034,0.033,0.035,0.038,0.042,0.045,0.052,0.062,0.073,0.082,0.096,0.12,0.14,0.16,0.19,0.24,0.29
4,,,,,0.026,0.027,0.029,0.032,0.035,0.041,0.048,0.057,0.064,0.074,0.091,0.11,0.13,0.15,0.18,0.22
5,,,,,,0.019,0.02,0.023,0.024,0.028,0.033,0.039,0.044,0.052,0.063,0.076,0.087,0.1,0.13,0.15
6,,,,,,,0.014,0.015,0.017,0.019,0.023,0.027,0.03,0.035,0.043,0.052,0.06,0.071,0.087,0.11
7,,,,,,,,0.01,0.011,0.013,0.015,0.018,0.021,0.024,0.029,0.035,0.04,0.048,0.059,0.072
8,,,,,,,,,0.0076,0.0089,0.01,0.012,0.014,0.016,0.02,0.024,0.028,0.033,0.04,0.049
9,,,,,,,,,,0.0061,0.0072,0.0085,0.0096,0.011,0.014,0.016,0.019,0.022,0.027,0.033


In [49]:
tlevels=Tree.interest_tree.levels
ttlevels=[]
for i in tlevels:
    temp=[]
    for ii in i:
        temp.append(ii.rate)
    ttlevels.append(temp)

ann_compounded=[]
mat=1/4
for i in ttlevels:
    temp=[]
    for j in i:
        temp.append(compounded_from_discount(discount_from_continuous(j,mat),mat,1))
    ann_compounded.append(temp)
    mat+=1/4

tdf=pd.DataFrame(ann_compounded).T
tdf.columns=Tree.periods[1:]
tdf

,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
0,0.053243,0.056252,0.058041,0.059446,0.058359,0.061691,0.066663,0.074663,0.080105,0.094133,0.111834,0.133899,0.151998,0.178705,0.223730,0.273648,0.321108,0.390009,0.500030,0.638087
1,NaN,0.047446,0.048948,0.050129,0.049216,0.052013,0.056184,0.062890,0.067447,0.079179,0.093949,0.112310,0.127330,0.149428,0.186511,0.227381,0.266012,0.321720,0.409812,0.518969
2,NaN,NaN,0.041308,0.042300,0.041533,0.043885,0.047389,0.053019,0.056842,0.066672,0.079025,0.094346,0.106850,0.125200,0.155873,0.189509,0.221145,0.266509,0.337652,0.424874
3,NaN,NaN,NaN,0.034131,0.033514,0.035403,0.038219,0.042737,0.045802,0.053675,0.063549,0.075765,0.085711,0.100270,0.124504,0.150939,0.175673,0.210929,0.265743,0.332196
4,NaN,NaN,NaN,NaN,0.025892,0.027346,0.029511,0.032984,0.035338,0.041377,0.048936,0.058267,0.065847,0.076913,0.095261,0.115175,0.133715,0.159990,0.200505,0.249097
5,NaN,NaN,NaN,NaN,NaN,0.018977,0.020473,0.022870,0.024494,0.028654,0.033851,0.040251,0.045437,0.052988,0.065458,0.078919,0.091387,0.108951,0.135798,0.167640
6,NaN,NaN,NaN,NaN,NaN,NaN,0.013943,0.015569,0.016671,0.019489,0.023006,0.027328,0.030825,0.035906,0.044272,0.053268,0.061568,0.073211,0.090896,0.111699
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010526,0.011269,0.013168,0.015535,0.018442,0.020790,0.024197,0.029796,0.035800,0.041326,0.049053,0.060738,0.074406
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007641,0.008927,0.010528,0.012491,0.014076,0.016375,0.020146,0.024183,0.027891,0.033067,0.040870,0.049963
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006119,0.007215,0.008558,0.009641,0.011212,0.013785,0.016538,0.019063,0.022582,0.027878,0.034033


## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

In [50]:
swap_rate

0.03668557489963309

In [51]:
coupPay=True
coupon=swap_rate
freq_b=4
face=100
rates=Tree.interest_tree
freq=1/(Tree.periods[1]-Tree.periods[0])
tprices=[]
historical=[]
for j in range(len(rates.levels[-1])):
    tprices.append((face*(1+(coupPay*coupon/freq_b)))*np.exp(-rates.levels[-1][j].rate*(1/freq)))
# coupPay= not coupPay
historical.append(tprices)
for j in range(2,len(rates.levels)+1):
    tkprices=[]
    for k in range(len(rates.levels[-j])):
        average=(tprices[k]+tprices[k+1])/2
        average+=coupPay*face*coupon/freq_b
        fact=np.exp(-rates.levels[-j][k].rate*(1/freq))
        tkprice=average*fact
        tkprices.append(tkprice)
    # coupPay= not coupPay
    tprices=tkprices
    historical.append(tprices)

historical=historical[::-1]
print('The price of the bond through the tree is {:.2f}'.format(tprices[0]))

The price of the bond through the tree is 98.77


In [52]:
tdf=pd.DataFrame(historical).T
tdf.columns=Tree.periods[1:]
tdf=100-tdf
tdf

,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
0,1.234054,3.154730,5.138814,7.218505,9.402792,11.744786,14.138552,16.535274,18.853657,21.122725,23.151437,24.845599,26.099771,26.954544,27.239523,26.622417,24.962294,22.190943,17.799550,10.796750
1,NaN,-1.430697,0.336679,2.198467,4.170165,6.307443,8.523814,10.783103,13.018937,15.262068,17.340089,19.163694,20.628718,21.757337,22.373785,22.157068,20.968535,18.747765,15.062805,9.097103
2,NaN,NaN,-3.728395,-2.086471,-0.345504,1.551302,3.534863,5.581470,7.639951,9.748166,11.758837,13.597607,15.173132,16.505987,17.430988,17.643126,16.997788,15.422660,12.530541,7.632152
3,NaN,NaN,NaN,-5.646040,-4.118885,-2.460170,-0.723503,1.076355,2.902546,4.795208,6.635549,8.366523,9.917267,11.313730,12.417423,12.963214,12.819321,11.905169,9.856793,6.065978
4,NaN,NaN,NaN,NaN,-7.119168,-5.666518,-4.152687,-2.586853,-0.996347,0.657716,2.283949,3.840432,5.273231,6.612745,7.756079,8.491239,8.714871,8.368384,7.152745,4.541223
5,NaN,NaN,NaN,NaN,NaN,-8.111025,-6.776266,-5.404366,-4.014412,-2.577472,-1.158340,0.213482,1.499654,2.723452,3.811524,4.610333,5.045548,5.076708,4.490979,2.918227
6,NaN,NaN,NaN,NaN,NaN,NaN,-8.630087,-7.398605,-6.155149,-4.879399,-3.616425,-2.386742,-1.217047,-0.090499,0.942078,1.773311,2.351883,2.651116,2.524503,1.719323
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.780658,-7.639014,-6.475981,-5.322730,-4.193578,-3.107070,-2.051593,-1.062023,-0.212107,0.463539,0.947926,1.142346,0.877374
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.658298,-7.571696,-6.492564,-5.430996,-4.399979,-3.391788,-2.430521,-1.566734,-0.823704,-0.212155,0.201841,0.305437
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.324588,-7.295085,-6.278211,-5.283152,-4.304831,-3.360143,-2.484098,-1.692596,-0.992553,-0.428556,-0.076321


## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

In [53]:
tdf

,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
0,1.234054,3.154730,5.138814,7.218505,9.402792,11.744786,14.138552,16.535274,18.853657,21.122725,23.151437,24.845599,26.099771,26.954544,27.239523,26.622417,24.962294,22.190943,17.799550,10.796750
1,NaN,-1.430697,0.336679,2.198467,4.170165,6.307443,8.523814,10.783103,13.018937,15.262068,17.340089,19.163694,20.628718,21.757337,22.373785,22.157068,20.968535,18.747765,15.062805,9.097103
2,NaN,NaN,-3.728395,-2.086471,-0.345504,1.551302,3.534863,5.581470,7.639951,9.748166,11.758837,13.597607,15.173132,16.505987,17.430988,17.643126,16.997788,15.422660,12.530541,7.632152
3,NaN,NaN,NaN,-5.646040,-4.118885,-2.460170,-0.723503,1.076355,2.902546,4.795208,6.635549,8.366523,9.917267,11.313730,12.417423,12.963214,12.819321,11.905169,9.856793,6.065978
4,NaN,NaN,NaN,NaN,-7.119168,-5.666518,-4.152687,-2.586853,-0.996347,0.657716,2.283949,3.840432,5.273231,6.612745,7.756079,8.491239,8.714871,8.368384,7.152745,4.541223
5,NaN,NaN,NaN,NaN,NaN,-8.111025,-6.776266,-5.404366,-4.014412,-2.577472,-1.158340,0.213482,1.499654,2.723452,3.811524,4.610333,5.045548,5.076708,4.490979,2.918227
6,NaN,NaN,NaN,NaN,NaN,NaN,-8.630087,-7.398605,-6.155149,-4.879399,-3.616425,-2.386742,-1.217047,-0.090499,0.942078,1.773311,2.351883,2.651116,2.524503,1.719323
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.780658,-7.639014,-6.475981,-5.322730,-4.193578,-3.107070,-2.051593,-1.062023,-0.212107,0.463539,0.947926,1.142346,0.877374
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.658298,-7.571696,-6.492564,-5.430996,-4.399979,-3.391788,-2.430521,-1.566734,-0.823704,-0.212155,0.201841,0.305437
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.324588,-7.295085,-6.278211,-5.283152,-4.304831,-3.360143,-2.484098,-1.692596,-0.992553,-0.428556,-0.076321


## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

In [54]:
terminal=historical[4].copy()
terminal=[max(100-i,0) for i in terminal]

tlevels=Tree.interest_tree.levels
ttlevels=[]
for i in tlevels[:4]:
    temp=[]
    for ii in i:
        temp.append(ii.rate)
    ttlevels.append(temp)

swap_tree=[]
swap_tree.append(terminal)
for i in range(1,len(ttlevels)+1):
    temp=[]
    for j in range(len(terminal)-1):
        average=(terminal[j]+terminal[j+1])/2
        daverage=average*np.exp(-ttlevels[-i][j]*(1/freq))
        temp.append(daverage)
    terminal=temp
    swap_tree.append(terminal)

swap_tree=swap_tree[::-1]
tdf=pd.DataFrame(swap_tree).T
tdf.columns=Tree.periods[1:6]
tdf

,0.25,0.50,0.75,1.00,1.25
0,1.545902,2.629204,4.313207,6.689209,9.402792
1,NaN,0.502957,1.017640,2.059741,4.170165
2,NaN,NaN,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,0.000000,0.000000
4,NaN,NaN,NaN,NaN,0.000000


## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

In [55]:
print('Blacks:', Swap0)
print('Tree:', swap_tree[0][0])

Blacks: 1.6857037328339315
Tree: 1.5459022979710222


## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

In [56]:
terminal=historical[4].copy()
terminal=[max(100-i,0) for i in terminal]

callable=[]
callable.append(terminal)
for i in 

SyntaxError: invalid syntax (3123280055.py, line 6)

***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

## 3.2

Price the **american** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

***